In [25]:
# Importing necessary modules
import pandas as pd
import nltk
import re
import random
from heapq import nlargest

In [2]:
# Setting up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Reading in and processing the gg2013 tweets file
tweets = pd.read_json('gg2013.json')
for i in range(0, len(tweets)):    
    tweets.at[i, 'text'] = re.sub("^RT @[a-zA-Z0-9_]*: ", "", tweets.loc[i]['text']).lower()
tweets

,text,user,id,timestamp_ms
0,jlo's dress! #eredcarpet #goldenglobes,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,what's making sofia vergara's boobs stay like ...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,kerry washington is everything. dying over her...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,anne hathaway has got me living. #goldenglobes,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38
4,jennifer lopez's lace dress? thoughts? #golden...,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38
...,...,...,...,...
174638,i was sad that mandy patinkin didn't win #gold...,"{'screen_name': 'dana1204', 'id': 18091543}",290675889379876864,2013-01-14 04:25:07
174639,jennifer lawrence aceptando su premio #goldeng...,"{'screen_name': 'IamTrisEverdeen', 'id': 55126...",290675889128230914,2013-01-14 04:25:07
174640,"golden globes, lots of fashion messes...but gl...","{'screen_name': 'Dpharmakis23', 'id': 852045842}",290675893024747523,2013-01-14 04:25:08
174641,did they have mug shots at the golden globes?!...,"{'screen_name': 'reynaramirez22', 'id': 22732662}",290675888763314178,2013-01-14 04:25:08


In [26]:
# Searching tweets for the specified regular expression
possible_award_tweets = [] # List of possible tweets mentioning awards
for j in range(0, len(tweets)):
    text = tweets.loc[j]['text']
    if re.search("wins best", text) or re.search("nominated for best", text):
        possible_award_tweets.append(text) # Save a tweet if it contains "wins best" or "nominated for best"

# Splitting the possible tweets mentioning awards on best, and keeping only the right-hand side
possible_award_names = []
for award in possible_award_tweets:
    split_award = award.split("best")
    split_award_rhs = split_award[1]
    split_award_rhs = "best" + split_award_rhs # Appending 'best' back to the beginning of the right-hand side
    # Cleaning up the possible award names by taking some extra stuff out
    split_award_rhs = re.sub(" for .*", "", split_award_rhs) # Deleting everything that falls after the word "for", which is usually the winner name
    split_award_rhs = re.sub("#.*", "", split_award_rhs) # Deleting everything that falls after a hashtag, since these are usually after the award names
    split_award_rhs = re.sub("@.*", "", split_award_rhs) # Deleting everything that falls after an @
    split_award_rhs = re.sub("\.|!.*", "", split_award_rhs) # Deleting everything that falls after a period or exclamation point, since these are after award names
    split_award_rhs = re.sub(" at .*", "", split_award_rhs) # Deleting everything that falls after "at ", since that is usually followed by the ceremony name
    split_award_rhs = re.sub("http.*", "", split_award_rhs) # Deleting all web addresses
    split_award_rhs = re.sub(",", "-", split_award_rhs) # Changing all commas to dashes
    split_award_rhs = re.sub("-$", "", split_award_rhs) # Deleting all dashes that end lines
    split_award_rhs = split_award_rhs.strip() # Deleting leading and trailing spaces
    possible_award_names.append(split_award_rhs)

# Counting the occurrences of all possible award names
possible_award_names_count = {}
for award in possible_award_names:
    if award not in possible_award_names_count:
        possible_award_names_count[award] = 1
    else:
        possible_award_names_count[award] += 1

# Keeping the top 20 mentioned award names
awards = nlargest(20, possible_award_names_count, key = possible_award_names_count.get)
print(awards)

['best supporting actress', 'best actor', 'best director', 'best original song', 'best actress', 'best supporting actor', 'best actor in a motion picture- comedy or musical', 'best song', 'best drama', 'best motion picture- musical or comedy', 'best motion picture- comedy or musical', 'best picture- musical or comedy', 'best motion picture- comedy or musical congrats to the cast and crew', 'best screenplay', 'best picture', 'best picture- drama', 'best actor- drama', 'best actress in tv drama- rubs in her', 'best actress in a drama', 'best motion picture drama']


In [13]:
# Adding the award name to the 'answers' DataFrame
answers['award'] = awards
answers